@context: UCU2023

In [1]:
import torch
torch.cuda.is_available()

True

## Preliminaries

In [9]:
from typing import Dict

import os
import numpy as np
import pandas as pd
import random
from collections import Counter

from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer

import spacy 
nlp = spacy.load("en_core_web_sm")


## Load datasets

In [3]:
path_data = '../../data'

In [4]:
dataset = load_dataset("csv", 
                       data_files={"train": f'{path_data}/train.csv', 
                                   "val": f'{path_data}/val.csv'});

Found cached dataset csv (C:/Users/Admin/.cache/huggingface/datasets/csv/default-046d41c2076f8a1d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
df_val = pd.DataFrame(dataset['val'])
df_val 


,text,words,pos,sentiment,tense
0,"X can use any attack , Poké - Power , or Poké ...",19,NOUN AUX VERB DET NOUN PUNCT PROPN PUNCT PROPN...,neutral,future
1,There were periods when everything was more op...,14,PRON VERB NOUN SCONJ PRON AUX ADV ADJ PUNCT CC...,neutral,past
2,Trans people are terrified of what lies ahead .,9,PROPN NOUN AUX VERB ADP PRON VERB ADV PUNCT,negative,present
3,Another Global horror is The Cube in Birmingha...,23,DET ADJ NOUN AUX DET PROPN ADP PROPN PUNCT DET...,negative,present
4,The innovative one piece stainless steel shiel...,26,DET ADJ NUM NOUN ADJ NOUN VERB NOUN ADP PROPN ...,positive,present
...,...,...,...,...,...
11027,The application of Dye Solar Cells ( DSC 's ) ...,38,DET NOUN ADP PROPN PROPN PROPN PUNCT PROPN PAR...,positive,present
11028,Along with the majority of Canadians elsewhere...,22,ADP ADP DET NOUN ADP PROPN ADV PUNCT PROPN VER...,neutral,past
11029,"IBM , like other established tech companies , ...",33,PROPN PUNCT ADP ADJ VERB NOUN NOUN PUNCT VERB ...,positive,past
11030,Valuation movements ( think potential falls ) ...,23,NOUN NOUN PUNCT VERB ADJ NOUN PUNCT AUX NOUN V...,neutral,future


## Additional functions

In [10]:
ADDITIONAL_TOKENS = {
    'WORDS': '<COND_LENGTH>',
    'POS': '<COND_POS>',
    'SENTIMENT': '<COND_SENT>',
    'TENSE': '<COND_TENSE>',    
    'START': '<START>',
    'END': '<END>',
    'PAD': '<PAD>'
    }  

In [11]:
def create_prompt(conditions: Dict[str, str], train_txt: str = None) -> str:

      """
      This function takes a dictionary of conditions and an optional training text, and constructs a prompt string
      by combining the conditions and training text with appropriate additional tokens.

      Args:
         conditions (Dict[str, str]): A dictionary of conditions where keys are condition names and values are condition values.
         train_txt (str, optional): An optional training text to be included in the prompt string.

      Returns:
         str: The generated prompt string.

      Example:
         prompt = create_prompt({"len": 10, "pos": "DET NOUN ADP PROPN", "sent": "neutral", "tense": "present"}, train_txt="Write a short article about sports.")

      Note:
         The function assumes the 'ADDITIONAL_TOKENS' dictionary is defined and contains tokens for the condition names.
      """

      # check that we know every condition
      assert all([(cond_name.upper() in ADDITIONAL_TOKENS) for cond_name in conditions.keys()])
      
      # convert every value to str
      conditions = {cond_name:str(cond_value) for cond_name, cond_value in conditions.items()}

      prompt = ''
      for cond_name, cond_value in conditions.items():
         prompt += ADDITIONAL_TOKENS[cond_name.upper()] + ' ' + cond_value + ' '
      prompt += ADDITIONAL_TOKENS['START'] 

      if train_txt is not None:
           prompt += ' ' + train_txt + ' ' + ADDITIONAL_TOKENS['END']
           
      return prompt 

In [8]:
def generate(model: GPT2LMHeadModel, tokenizer: GPT2Tokenizer, conditions: Dict[str, str]):
    
    prompt_text = create_prompt(conditions)

    if 'len' in conditions:
        expected_len = int(conditions['len'])
    elif 'pos' in conditions:
        expected_len = len(conditions['pos'].split())
    else:
        expected_len = 30

    # encode promt
    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to("cuda")

    # generate
    output_sequences = model.generate(
        input_ids=encoded_prompt,
        pad_token_id=tokenizer.eos_token_id,
        max_length=expected_len + len(encoded_prompt[0]) + 10,
        temperature=1.0,
        top_k=0,
        top_p=0.9,
        repetition_penalty=1.0,
        do_sample=True,
        num_return_sequences=1,
    )

    # decode text
    generated_sequence = output_sequences[0].tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # find the boundaries
    end_index = text.find("<END>") if "<END>" in text else 1000
    pad_index = text.find("<PAD>") if "<PAD>" in text else 1000
    fin_index = len(text)
    stop_index = min([end_index, pad_index, fin_index])
    str_out = text[len(prompt_text) + 1:stop_index].strip()

    return str_out    

In [9]:
def find_attributes(input_str: str) -> dict:
    spacy_doc = nlp(input_str)
    return {'words': len(spacy_doc), 'pos': ' '.join([t.pos_ for t in spacy_doc])}

In [12]:
print(f'* train:\n{create_prompt({"words": 7, "sentiment": "neutral", "tense": "present"}, train_txt="Write a short article about sports.")}')
print()
print(f'* just prompt:\n{create_prompt({"words": 7, "sentiment": "neutral", "tense": "present"})}')

* train:
<COND_LENGTH> 7 <COND_SENT> neutral <COND_TENSE> present <START> Write a short article about sports. <END>

* just prompt:
<COND_LENGTH> 7 <COND_SENT> neutral <COND_TENSE> present <START>


## Model (generation)

In [35]:
#model_path = '/mnt/green-efs/andrii.liubonko/may/ucu/gpt2/exp0013/epoch-checkpoint-4.0'
model_path = '../../models/len/epoch-checkpoint-4.0'
model = GPT2LMHeadModel.from_pretrained(model_path)
model.to('cuda');

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token=ADDITIONAL_TOKENS['START'], eos_token=ADDITIONAL_TOKENS['END'], pad_token=ADDITIONAL_TOKENS['PAD']) 
tokenizer.add_special_tokens({'additional_special_tokens': [ADDITIONAL_TOKENS[t] for t in ADDITIONAL_TOKENS.keys() if t not in ['START', 'END', 'PAD']]})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


4

### Generation attributes

In [33]:
df_train = pd.DataFrame(dataset['train'])

In [34]:
words_number = df_train['words'].to_numpy()[np.random.randint(1, df_train['words'].to_numpy().shape, 2000)]

### len / sentiment / tense

In [37]:
# current_conditions = {'sentiment': 'positive'}
# current_conditions = {'tense': 'future'}
# current_conditions = {'words': 5, 'sentiment': 'positive'}
# current_conditions = {'words': 5, 'tense': 'present'}
# current_conditions = {'sentiment': 'neutral', 'tense': 'present'}
# current_conditions = {'words': 10, 'sentiment': 'positive', 'tense': 'future'}

generated_sentences = []
control = []

for i in range(2000):
    # = set condition
    current_conditions = {'words': words_number[i]}
    # if i <= 332: tense = 'past'
    # elif i >= 667: tense = 'future'
    # else: tense = 'present'
    # current_conditions = {'tense': tense}
    # = generation
    generated_str = generate(model, tokenizer, current_conditions)
    # = report
#    print(current_conditions)
    # print(i)
    print(generated_str)
    generated_sentences.append(generated_str)
    control.append(words_number[i])
#    print(find_attributes(generated_str))
    print('\n')

It can not be read as this story ends, and must be read as Mr. King and/or rather as this story unfolds, in any case, as it stands at the end of the most


" Ms Collins said she was told when she woke she saw that she had been shot in the throat.


As they ’d gone into the thickets of pictures like this, why did n’t the Obama administration have the courage to bring out an American after the 9/11 panel, which


Ties that have been around for decades have proliferated as cars and trucks were added to the world's roads, with the Transportation Security Administration standing down.


Ultimately, starting the Giants ’ offseason practice on June 18 would be a good idea, since Ryan Griffin would likely still be in the squad in Kansas City next summer.


Perhaps the best examples of this phenomenon are the oft - ignored PISA data that make this type of revelation.


The new unit, however, is particularly impressive : the looks are unmistakably gorgeous ; and the glass of the new unit are rus

Bike lanes, now in Hampshire and in Massachusetts, could technically operate as roads on privately owned land along the Green Line.


In May, the capital, Russia, unveiled the worst cold weather on record, with 2.5 and 3.5 C on record at worst, respectively, in January and February, respectively, according


He picked his new co - host, Don White, on Saturday night to see them tackle questions from some of their biggest supporters : well, not some of the most famous names on the show : How


The other guy in the circle of two took him up and blew him up.


A recent poll by Pew Research found that 49 percent of U.S. adults still believed their own policies should be consistent.


After reaching the summit in November 1944, Captain Bonenberger flew by the air to photograph the new Soviet Union from the space station's submarine, Chang Schooft.


The extent to which the industry loses users, to which Apple is taking an extraordinary step in its corporate support for Supermicro, is strikin

The voters here will decide whether they want to keep Sheikh Miskeen, the biggest town in North Waziristan.


It's good for him to fight so in the ring.


He tells me this one : the FBI director has the right to determine who is behind the hacks : “ Once we have the truth, we have to assess what, if anything, means the �


Just as there is n’t any precedent for the new crackdown, everyone supports.


" At that stage the conference will be pretty dramatic, " she said in the briefing.


Slavery was translated as the work of a pamphlet by Nicholas de Soto, who wrote the following : “ The Slaves Without Slaves : The Use of Women to Execute Slaves for


New Delhi ( CNN ) - Japan is launching a new fast - growing Internet of Things ( PLAN ) service to promote the usage of 21st - century Baidu access networks across the country, a


“ Please, Vladimir Putin, we have to consider, the increasing costs of war will be driving some very expensive, wasteful spending programmes into the ground, ” sa

But it has become more and more difficult to see how much other parts of the world are impressing the Communist masses.


But I can't argue with the 5 men. "


B ) Being an adult does not mean being an adult has nothing to do with learning.


" In many years, as horrific and senseless as the United States'wholesale military interventions have become in places such as Dover, it has become an important national memory. "


Lately, governments and settlers have shown a willingness to react with “ greening ” strategies, particularly when it comes to the environment and the health of rivers and streams.


But our plan is no doubt an amazing change : drop a few dead, instead of protecting & eating a couple of more.


I think it ’s an essential part of your continued mental health.


Sign up for Take Action Now and we ’ll send you three meaningful actions you can take each week.


Instead, he used the use of his work to expand upon the original script and made the two - part finale, making it

With the PlayStation 3 adding Back support for both consoles, PS4 and PS4 owners will have the option to purchase additional content from the third party, a retail release game store for your purchase, or


In other words, the poor will not understand basic necessities and will suffer under the subsidies.


The Burmese seized all kinds of fish and killed it.


There ’s a lot of guys out there that just do n’t.


Another rebel group's president, Nouri al - Maliki, refused to recognize any separate figures in an interview with Reuters.


He ’s serving a 10th term on the Senate Judiciary Committee, and currently has become the longest - serving member of the House Intelligence Committee, serving the 9th Congressional District, acting on a


Her campaign has also been a huge success, with some claiming to have made more than $ 200,000 in ads this year.


And no one should see their value in the content that the creators are going to pay for.


The upshot is that it is just getting richer, 

...

Therefore, this should not have been the case. ”


The frustrating reality is that people still care very little about the opinions of their leaders and they do n’t want to hear from them.


Packed with low price Reynolds will probably drive you crazy.


If you insist upon suspending the liberty of one ’s " conscious self - awareness " to future Nazis, these policies will invariably violate one ’s fundamental human rights.


Users interested in accessing Viber can choose to join the growing community, simply by visiting the MViber website and following the link.


During pre - authorization, users can request a shared object or query from the configured CRUD model.


In other words, a cluster of " members " would fit within the definition of a cluster of black holes, giving rise to the hypothetical astrophysicidic and black holes in our universe, and


I used to be just as superstitious and nostalgic for yukko, but now I hope this trade will happen.


Overall, The Register read : “ As 

That's your turn to “Try More! ”


I definitely agree with Jack that there is a lot of room for improvement. ”


“ We do n’t expect to see either of those amenable, ” she said of post-2005 global trade stimulus spending, in a trade buzzword.


Everybody is looking for fixes too … just like Ryan.


It also raised questions about feasibility and given the ambiguity of the project it has been impossible to build the station itself properly.


Unfortunately, we found a number of interesting designs of boats which are quite common to think about.


Why was she mutilating you?


The shelter's population has risen over a four - year period. "


Quite a couple things here, although it's never quite possible to pull that off.


He smiles and laughs with a laugh on the cuff.


Further information on the 787 victims was provided by the victims ’ family members and friends, the Los Angeles Times reported.


The JIAAC analysed the food deaths and injuries suffered after an incident in which Malaysi

This was one of the few reasons that Kennedy made progress on tax reform in the Senate.


It would have been more enjoyable and cheaper for him to have gotten surgery done. "


He must face charges in connection with a hate speech and get away with it.


It was important.


That ’s OK, I do have some very important books available right now. ”


After artist Noah Hawley was creative with maps for Arkans to draw, the seaside town of Black Sands was re - imagined as a place where ships and artillery gunships are once again.


It is because of this little, yellow type of chip that most people ignore the taste of.


After officers complained, police said they were called to see him as someone else and denied him.


At first my job was to survive.


This was not an ideal situation for Bengal, because the military considered it an important occupation to provide a necessary safety for some of the region.


They are not super special measures because they simply are not doing something right 

By 2010, thanks to rising inequality in the US and among the world's most affluent, Americans had more access to government services and health, compared to a bottom of 32 percent in the developed world


He ’s a big fan of the show, so it makes him an ideal target.


The Xbox One X will be released in Japan on November 27th, alongside the PS4, Xbox 360 and PC at MES.


This article is going to get old, but I am sure that's fine and good.


The biggest draw for Microsoft's Windows 10 operating system, the Surface Pro, is the new Surface Pro 4, released in November, and its price has exploded in the past month.


ärvingprazku said that Poland ’s GDP was less than 7 percent above its EU average ( more than 12 percent below EU average ).


James Carr ’s 40 - yard run game against Green Bay is the clear highlight.


Should I adopt a version of the iPhone app or attempt to live on Amazon?


“ Let ’s not get the idea of a million people that are trapped in three different versions of a massi

No telescope will ever measure the difference in how much light we are able to measure by looking at it, unless in all of these thousands of years we can make a determination.


'In other words, we could not expect to see a monopoly of trade or commerce.


However, he is simply not the best skater and has the power to do.


It was like I was stuck in the other side of my brain.


" Emotional trauma, " she added, " hurt feelings, " depression and anxiety. "


Other foreign policy concerns could also be viewed as destabilizing or dangerous.


" I wanted to come in to talk with my dad about this, and he ’s amazing, " Rosario said. "


Q : You can't deny the impact of wearable devices on your personal space.


A thin tower surrounded by an old woman's bed was broken, then there was a pile of books stacked on top of her scroll, offering to explain the patterns of time.


“ This kind of digital imaging system has about 50 times less bandwidth than LCD.


Still, House Republicans refused to r

In [38]:
df_len2000 = pd.DataFrame({'text': generated_sentences, 'length': control})
df_len2000

,text,length
0,"It can not be read as this story ends, and mus...",46
1,""" Ms Collins said she was told when she woke s...",19
2,As they ’d gone into the thickets of pictures ...,59
3,Ties that have been around for decades have pr...,28
4,"Ultimately, starting the Giants ’ offseason pr...",31
...,...,...
1995,"At another point, he said : “ That sounds like...",24
1996,The US invaded the Hague Convention of Human R...,10
1997,Deep learning approaches have successfully fos...,20
1998,These films will feature 16 - bit cinematograp...,26


In [40]:
df_len2000.to_csv('../../../Controllable-text-gen-with-DM/datasets/Generation_results/df_len2000.csv', index=False)

### pos

In [12]:
# preparation
samples = Counter(df_val['pos']).most_common()
samples[:3]

[('PRON VERB ADJ PUNCT', 11),
 ('PRON AUX VERB PRON ADJ NOUN PUNCT', 10),
 ('PRON AUX ADJ PUNCT', 9)]

In [18]:
len(samples)

10863

In [30]:
generated_sentences = []
control_pos = []
control_sentiment = []
control_tense = []

for i in range(1000):
    # set condition
    sample_index = i
    if i <= 332: sentiment = 'negative'
    elif i >= 667: sentiment = 'positive'
    else: sentiment = 'neutral'
    tense = random.choice(['past', 'present', 'future'])
    # current_conditions = {'pos': samples[sample_index][0]}
    current_conditions = {'pos': samples[sample_index][0], 'sentiment': sentiment, 'tense': tense}
    # generation
    generated_str = generate(model, tokenizer, current_conditions)
    # report
    # print(f'CONDITIONS   : {current_conditions}')
    print(i, current_conditions)
    print(generated_str)
    generated_sentences.append(generated_str)
    control_pos.append(samples[sample_index][0])
    control_sentiment.append(sentiment)
    control_tense.append(tense)
    # print(f'RESULT ATTR. : {find_attributes(generated_str)}')
    print('\n')

0 {'pos': 'PRON VERB ADJ PUNCT', 'sentiment': 'negative', 'tense': 'present'}
It ’s useless.


1 {'pos': 'PRON AUX VERB PRON ADJ NOUN PUNCT', 'sentiment': 'negative', 'tense': 'past'}
They had lost their entire expedition.


2 {'pos': 'PRON AUX ADJ PUNCT', 'sentiment': 'negative', 'tense': 'past'}
It was painful.


3 {'pos': 'PRON AUX VERB PUNCT', 'sentiment': 'negative', 'tense': 'present'}
It is painted.


4 {'pos': 'VERB ADP PRON AUX VERB NOUN VERB NOUN NOUN PUNCT NOUN CCONJ NOUN ADP DET PROPN PROPN PROPN PUNCT', 'sentiment': 'negative', 'tense': 'future'}
Based on this will hurt business lending institutions, investors and members of the Federal Reserve System.


5 {'pos': 'PRON AUX VERB PRON PUNCT', 'sentiment': 'negative', 'tense': 'future'}
You must hate that.


6 {'pos': 'PRON AUX VERB DET NOUN PART VERB ADP PUNCT', 'sentiment': 'negative', 'tense': 'future'}
They will refuse the plan to give up.


7 {'pos': 'PRON VERB DET ADJ NOUN PUNCT', 'sentiment': 'negative', 'tense': 'pre

67 {'pos': 'NUM PROPN PROPN PUNCT NUM VERB DET NOUN ADP ADV PUNCT VERB NOUN CCONJ AUX VERB ADP NOUN ADV CCONJ ADV PUNCT', 'sentiment': 'negative', 'tense': 'past'}
19 Tomoe River, 18 died this year from long - acting pneumopathology and was kept in place long and arduously.


68 {'pos': 'PRON ADV VERB ADJ PUNCT', 'sentiment': 'negative', 'tense': 'present'}
It just gets worse.


69 {'pos': 'DET ADJ NOUN AUX DET NOUN PUNCT', 'sentiment': 'negative', 'tense': 'present'}
This bad guy is a crook.


70 {'pos': 'PRON AUX ADV VERB ADP DET NOUN PUNCT', 'sentiment': 'negative', 'tense': 'past'}
He was heavily disbarred by the court.


71 {'pos': 'PRON AUX AUX VERB PUNCT', 'sentiment': 'negative', 'tense': 'past'}
They had been cheating.


72 {'pos': 'PRON ADV AUX DET ADJ NOUN PUNCT', 'sentiment': 'negative', 'tense': 'past'}
That alone was a huge risk.


73 {'pos': 'DET ADJ NOUN AUX VERB ADP PROPN PUNCT', 'sentiment': 'negative', 'tense': 'present'}
A second attack is scheduled for Wednesday.



105 {'pos': 'PRON AUX VERB DET ADJ NOUN PUNCT ADV PRON AUX PART ADV AUX ADV VERB PUNCT PUNCT PRON VERB PUNCT', 'sentiment': 'negative', 'tense': 'present'}
I have made this clear today, however I do not actually read, ” she says.


106 {'pos': 'SCONJ PRON AUX ADJ PART VERB SCONJ NOUN AUX DET NOUN PUNCT VERB NOUN PUNCT DET NOUN AUX ADJ ADP ADJ NOUN PUNCT', 'sentiment': 'negative', 'tense': 'past'}
If we were able to understand that fraud was a cover - breaking bug, the result was immediate for many people.


107 {'pos': 'PUNCT PROPN PROPN ADV VERB PUNCT PROPN PUNCT PUNCT', 'sentiment': 'negative', 'tense': 'present'}
" Mind Geo now means " Fuck "!


108 {'pos': 'DET PROPN AUX AUX VERB ADP PRON PUNCT PUNCT', 'sentiment': 'negative', 'tense': 'future'}
The Muslims must be ashamed of themselves. ”


109 {'pos': 'NOUN AUX VERB PROPN ADJ NOUN ADJ PROPN PUNCT SCONJ DET PROPN PROPN VERB SCONJ ADJ NOUN VERB NUM NOUN ADP PROPN ADP DET ADJ NOUN PUNCT', 'sentiment': 'negative', 'tense': 'present'}

133 {'pos': 'NOUN ADP DET PROPN CCONJ ADP DET NOUN NOUN PUNCT PROPN PUNCT AUX ADJ ADP NOUN CCONJ NOUN ADP DET ADJ NOUN SCONJ ADP DET NOUN ADP DET ADJ NOUN PART PRON DET NOUN VERB PUNCT', 'sentiment': 'negative', 'tense': 'future'}
Children at the Heimwehr and at the movie theater, Universal, are subject to reputations and criticism from the political class except in the name of the courageous individuals to whom that criticism stands.


134 {'pos': 'PUNCT ADV VERB PROPN PUNCT PUNCT NOUN ADP PRON NOUN AUX AUX VERB VERB ADP NOUN ADP NOUN CCONJ ADP NOUN PART NOUN PUNCT', 'sentiment': 'negative', 'tense': 'future'}
" Instead leading Republicans, " half of their voters will be giving back money in debt to balance and in candidates'run.


135 {'pos': 'PRON AUX DET ADJ NOUN PART VERB PRON PROPN PROPN CCONJ PROPN PROPN ADV ADJ PUNCT CCONJ VERB ADV ADP ADP DET NOUN PUNCT', 'sentiment': 'negative', 'tense': 'past'}
It was the perfect opportunity to change our Pledge Monday and Pledge Wednesday a

159 {'pos': 'ADP PRON PUNCT DET NOUN AUX ADV PROPN PUNCT DET NOUN PART VERB ADJ NOUN ADP DET NOUN ADP ADJ NOUN PUNCT DET NOUN DET PUNCT ADJ ADJ ADJ NOUN PUNCT VERB PRON DET ADJ NOUN ADP DET NOUN PUNCT', 'sentiment': 'negative', 'tense': 'past'}
For that, the retaliation was perhaps instinctive : the borrower to see Christian values as a sign of good faith ; the devil the “ greatest evil atheist friend ” gave him a free life after the onslaught.


160 {'pos': 'DET NOUN AUX VERB ADP SCONJ PART VERB DET NOUN PUNCT', 'sentiment': 'negative', 'tense': 'past'}
The injury was ruled out because not having the words.


161 {'pos': 'VERB DET ADJ NOUN ADP DET NOUN ADP NOUN PUNCT PRON VERB ADJ SCONJ ADJ NOUN AUX VERB NOUN VERB ADP ADJ NOUN NOUN PUNCT', 'sentiment': 'negative', 'tense': 'future'}
Based the absolute size of the numbers of landlords, it ’s clear that rental properties will discourage tenants renting on mixed use lots.


162 {'pos': 'ADV PUNCT PRON AUX VERB DET NOUN NOUN PRON VERB DET

...

872 {'pos': 'VERB ADJ NOUN PUNCT ADJ ADP ADJ CCONJ NOUN VERB DET NOUN ADP VERB NOUN DET NOUN PUNCT', 'sentiment': 'positive', 'tense': 'past'}
Wearable tech : better than most but others advanced the process of making applications a breeze.


873 {'pos': 'PRON AUX VERB PRON VERB ADP DET NOUN ADV PUNCT', 'sentiment': 'positive', 'tense': 'future'}
I 'd recommend you check out these models here.


874 {'pos': 'INTJ PUNCT PRON PRON VERB PRON ADJ NOUN NOUN ADV VERB ADP PRON NOUN PUNCT CCONJ NOUN ADP PRON NOUN ADP ADJ NOUN VERB ADP DET NOUN SCONJ PRON AUX NOUN VERB ADP DET NOUN ADP DET NOUN PUNCT PRON VERB DET NOUN PUNCT ADP DET NOUN VERB PRON ADJ NOUN ADP DET NOUN PUNCT', 'sentiment': 'positive', 'tense': 'past'}
Well, those who tell you average daily audience Members then look at your performance, and people in your audience like powerful voice acting in the name that they do n’t feel like a part of the audience, you see the moment, like a kid playing his own role in the movie.


875 {'po

900 {'pos': 'PRON VERB PART DET ADJ NOUN ADP DET NOUN PUNCT DET NOUN VERB PUNCT', 'sentiment': 'positive', 'tense': 'future'}
There ’s not a single song in the world, no one booing.


901 {'pos': 'PROPN PROPN PUNCT NOUN ADP ADJ NOUN ADP DET PROPN PROPN ADP PROPN CCONJ PROPN PROPN CCONJ NOUN ADP DET PROPN ADP PROPN PUNCT VERB ADJ NOUN NOUN PUNCT', 'sentiment': 'positive', 'tense': 'past'}
Michael Lewis, principal at public affairs for the National Government of Ontario and Finance Canada and author of the Pesticides of Agriculture, analysed statistical model analysis.


902 {'pos': 'ADV PRON VERB ADV PUNCT', 'sentiment': 'positive', 'tense': 'future'}
Hopefully we do better.


903 {'pos': 'PRON VERB ADJ NOUN PART VERB PUNCT ADP DET NOUN PUNCT DET PROPN PUNCT ADJ ADP VERB NOUN PUNCT NOUN VERB DET NOUN PUNCT ADJ NOUN PUNCT PUNCT PUNCT', 'sentiment': 'positive', 'tense': 'future'}
It ’s important decisions to vote, from a position, a conscience, such as winning campaign, that help the orga

930 {'pos': 'ADP DET NOUN PUNCT NOUN VERB PROPN AUX PART VERB ADJ NOUN ADP ADJ NOUN PART VERB DET NOUN NOUN PUNCT', 'sentiment': 'positive', 'tense': 'future'}
As a result, companies hoping Facebook can not get enough revenue to buy the revenue share.


931 {'pos': 'PRON AUX AUX ADV AUX NOUN CCONJ NOUN CCONJ NOUN PUNCT', 'sentiment': 'positive', 'tense': 'future'}
They can be seasonally be leaders and strategists and strategists.


932 {'pos': 'PROPN PUNCT PROPN PUNCT PUNCT NOUN ADP PROPN AUX VERB ADP DET NOUN NOUN CCONJ VERB NUM NOUN ADP VERB SCONJ VERB ADJ NOUN NOUN AUX AUX VERB ADP ADJ VERB NOUN NOUN PUNCT', 'sentiment': 'positive', 'tense': 'future'}
versatility ( Byf1 ), versatility of SATA will help in that scenario scenario and complement one strategy for avoiding that annoying SATA connection issues can be overcome by standard SATA based power flow.


933 {'pos': 'ADP DET NOUN ADP PROPN NOUN PUNCT DET NOUN AUX ADV ADV VERB PUNCT', 'sentiment': 'positive', 'tense': 'present'}
Fr

958 {'pos': 'PUNCT PRON AUX AUX ADV VERB ADP NUM PUNCT', 'sentiment': 'positive', 'tense': 'present'}
" She has been totally fulfilled by Jiggling.


959 {'pos': 'NOUN ADP PROPN VERB ADP DET NOUN PUNCT VERB PRON NOUN PUNCT CCONJ NOUN CCONJ NOUN SCONJ DET NOUN AUX VERB DET NOUN ADJ PUNCT', 'sentiment': 'positive', 'tense': 'present'}
Storylines from dialogue starting to the story, finding his enemies, and characters and vehicles as the story is telling a story new.


960 {'pos': 'AUX VERB ADV ADP ADP DET NOUN ADP NUM PUNCT PROPN ADP PROPN ADP NUM PROPN PUNCT PROPN AUX VERB ADP DET NOUN ADP PRON ADJ NOUN ADV NUM NOUN ADV ADP NUM PUNCT PROPN ADP PROPN PUNCT', 'sentiment': 'positive', 'tense': 'future'}
Being drafted third down in the draft by 31, Butler with NBA before 31 Feb., averaging will accelerate as a result of his outstanding play only 31 minutes away from 22:20, Butler with NBA.


961 {'pos': 'PROPN PROPN AUX AUX ADJ ADV PUNCT', 'sentiment': 'positive', 'tense': 'future'}
Chris C

989 {'pos': 'SCONJ PRON VERB ADJ NOUN PUNCT NUM PUNCT NOUN PUNCT ADJ PROPN PROPN VERB ADP NOUN NOUN ADJ PROPN NOUN ADP AUX VERB ADP NOUN CCONJ ADV VERB ADP DET PROPN PUNCT', 'sentiment': 'positive', 'tense': 'present'}
As you know earlier today, one - third - great Toronto Blue Jays fans around baseball know more Jays fans than do die for Sale or even die for the Yankees.


990 {'pos': 'ADP PUNCT PROPN NUM PUNCT NUM PUNCT NUM PROPN PROPN ADP PUNCT PROPN ADP PROPN NUM PUNCT NUM PUNCT NUM PROPN AUX DET VERB NOUN VERB ADJ NOUN PART VERB ADP DET NOUN PUNCT ADV DET NOUN VERB PRON PRON AUX VERB PUNCT PUNCT', 'sentiment': 'positive', 'tense': 'present'}
On [ March 27, 2013 ] 23 January 2010 ( 93 GMT+) from [ Martin in July 2007 ] 28th 2011 - 07 : 20 December 2010 - All thanks everyone who has contributed, " the statement reads that it has received ".


991 {'pos': 'PRON AUX PROPN VERB SCONJ DET ADV VERB NOUN AUX ADV ADV ADJ ADP NOUN PUNCT NOUN PUNCT PUNCT', 'sentiment': 'positive', 'tense': '

In [14]:
generated_sentences = []
control = []

for i in range(1000, 1200):
    # set condition
    sample_index = i
    for j in range(5):
        current_conditions = {'pos': samples[sample_index][0]}
        # current_conditions = {'pos': samples[sample_index][0], 'sentiment': 'neutral', 'tense': 'present'}
        # generation
        generated_str = generate(model, tokenizer, current_conditions)
        # report
        # print(f'CONDITIONS   : {current_conditions}')
        print(i)
        print(generated_str)
        generated_sentences.append(generated_str)
        control.append(samples[sample_index][0])
        # print(f'RESULT ATTR. : {find_attributes(generated_str)}')
        print('\n')

1000
( 2 ) This article originally appeared a project was abandoned due to a lack of valuable research the city ’s infrastructure and limited resources expended obsolete research, and was primarily a response to homeless people at the City.


1000
( c ) A claim otherwise treated the court may come up with a provision in third place that assents to coercion or other policies governing personal conduct, or is merely a defence of relevant law within the Constitution.


1000
[ SU - METAL ] The event also confirms the interest is getting in on the head in recent months the band ’s development and new label releases next week, and is now the focus of new acts by the Chordid.


1000
( 4 ) The attacker accidentally leaked this cable is passed over through a series of social networks whose sites and/or social networks redirect massive content, or is subsequently a target for massive protests in the US.


1000
( 2 ) The hypothesis also takes the temperature is eqptotically biased due to the inte

1011
When I opened this little little international system, many in the media dismissed vaccine safety concerns.


1012
I remember thinking something about the end of the Republican party's mainstream politics today, but the loss - coming repeal then coming back, brought back to the ground to my dismay when again and again almost every dropped that night, when that was kind of " cool " Trump, a skinny sitting still so casually.


1012
I like calling it off a day in a tragic way to baseball players, but the game ’s gotten here : getting back to the bench with your friends while still and still afterward you reach the field, when it is also open, and Sunday comes to, natural, refreshing, new, old, 365 days, the best feel no more.


1012
I enjoy seeing him as a player in the defensive back's booth booth, and this guy - mumbling guy really helped me, coming here with a bite of his crack after so and so long that ’s a gift, like I was really thirsty, and Loooong shut in, huge, intense, 2 mi

1025
Humans can breed tolerance.


1025
Details can form society.


1025
Obesity is driving changes.


1026
Clearly, you would have heard the news article about the marriage of the bitch.


1026
Besides, you can get the emp hacker in the ear of an ear.


1026
Unfortunately, this could have derailed the conversation timeline for the rest of the day.


1026
Surely, we must have fancied the computer community of all sectors of the world.


1026
Personally, I would have preferred a blank cover for the sides in a similar situation.


1027
They could use the fish again.


1027
You can watch the discussion below.


1027
He 'll keep that commitment though.


1027
You can see the signs here.


1027
You can read the transcript here.


1028
If you change the font in Hex ( you can't hide the change ).


1028
If you know any secret of Lohse … you can't tell any secret! "


1028
If they get a hit with Hootsuite, it won't affect the network. "


1028
While I had a chance at comedy, I didn't like the 

1038
Why is adultery considered sinful by religion?


1038
Why should Rangers fans be outraged about skater?


1039
I was intrigued, especially drawn to Wagner ’s theme park – there was something special about LA at that time.


1039
It was Ardublock, now known as Ardublock's home base, that became something akin to Pavlock in the 80s.


1039
He was delirious, intensely engaged in Sidney's affairs affairs ; she entralled him most at Sidney on the dunes.


1039
He was Kranz, always thinking about Whitewave ’s bankruptcy case, which made him bitter about Whitewave as a buyer.


1039
I do n’t, apparently fed up Latino ’s status, which gave me more than Crossus in that position.


1040
Download the images from Nikon 7 at the U.S. trade - it is HERE and up.


1040
Following the crisis of September 10 in the U.S. economy, it was welcomed and even in.


1040
Discussing the system at Lightyear 2015 in the L.A. forum, I can laugh and probably out.


1040
Following a successful Kickstarter campa

1054
During a visit to the Lincoln Bedroom, Spitzen described the scene in a letter to the Lincoln Bedroom on Wednesday in the Lincoln Bedroom on Wednesday, noting that militants and veterans bombed as a group of the enemy and a daring assault.


1054
Over the course of the day care, Chris made a presentation about the benefits of the Affordable Care Act in Philadelphia on the House of Lords, explaining that hospitals and health services supported by the community in both ways and the surrounding area.


1055
President Donald Trump is facing significant external pressure.


1055
Safety Car Harwood is offering full rear depth.


1055
Prince George ’s Theatre will announce new national roles.


1055
constado por hacesa ter eto meccamendiendi se pontificando.


1055
CLOSE Information National Park will receive additional public information.


1056
" Initially a Tribune - run " state - run " media was dubbed as news satire, " a 1995 Harvard economics professor at liberal Harvard economics 

...

1136
Pakistan banned the killing of senior militant Ammar al Haqqani ’s ( pictured March 20 ) Shuja Nawaz Sharif Ali alias of 38.


1136
Microsoft signed a deal with private Android Market Android studio, named Oizo already, Google Play Music Service partners in 2007.


1136
Pininfarina leads the list of bestsellers New York magazine cover stories, including Zoo Story : Born Free USA issue in 2008.


1136
Borussia Dortmund damaged the German byways through deadening Bundesliga Bundesliga Bundesliga Bundesliga engine, ending Bayern Munich easily, Old M RB Leipzig ’s 3 - 0 Champions


1137
It's extremely very very cool right, but really not the best of different.


1137
It's almost too close together, and just not the best of best.


1137
It was all very powerful early, and still not the least as painful.


1137
That is pretty much impossible currently, and probably not the best for most.


1137
It was very fun as well, and only not a bad for most.


1138
With the Stormtrooper win, the g

1148
Prosecutors have privately declined to ever allow trials like New Zealand ’s MH370, with the last one that costs $ 80,000 including the full price of a new Eurofighter jet.


1148
Broadcasters have repeatedly refused to directly answer questions about Mr Obama ’s Afghanistan, in the last 11 that cost $ 700,000 making the former president on a regular basis.


1148
SeaWorld is also reportedly planning to formally open stores in New Zealand ’s Christchurch, with the next one that ’s $ 100 m including a huge chunk of the international Disney space.


1148
Investigators have also worked to again link texts to Sinn Féin ’s iCloud, with the last one that suggests £ 1,930 worth a personal note on the new iPhone series.


1149
They picked up a quick phone to leave their destination but settled on something in L.A., and it worked well for them.


1149
I went to the medical school to get my mom and pop up some of Canada, and we talked really about it.


1149
We made with a little effort to 

1164
Feminists never spoke to Josh Richardson.


1164
Life also rejoiced in Adam Zappa.


1165
Thanks to Twitter, record shows and our readers can be coupon writers and Small Business owners.


1165
Items like Hanj, gold bars and your currency can be transferable or via courier service.


1165
Types like Ember, Fuel transport and our customisation could be code files or reactPublishing Monad.


1165
Thanks to Followers, team members and our readers has been rock solid and loyal contributors.


1165
Thanks to Fred, fashion braces and their sponsors can be fashion braces and C.J. accents.


1166
He had wounded the heart bit before, but luckily had run back home, informed of the news by a friend while he was on his third visit to hospital.


1166
I have researched this psychology question twice, but finally have settled on reasonably - established out the theory as a means for someone is at their own risk of happiness.


1166
You will fit the size measurements directly, and simply can tak

1178
The economic slowdown, led by weak and weak domestic production growth, and more attractive, added their middle class.


1178
A rich history, filled with cultural and intellectual valuable reading materials, and surprisingly inspiring, read their best work.


1178
The national media ( led by Ukrainian and American online media giants ) and then American - made their own deals.


1178
The primary Objective - Cucumber of malicious and unsuspecting human users, and especially vulnerable, makes their own statement.


1179
The FCC chairman will announce the timing of the Comcast of the ruling by a vote of FCC set to the convention next week.


1179
The UN panel will receive the award for the F2 as an example of the program during a meeting at Hampshire Hall called on the sporting Australian captain.


1179
The Red sources had assumed the death of the Cassini with the help of the press for Cassini launched as a base scientific mission.


1179
The U.S. military has launched an attack on 

1192
If you already have good intuition already some decent lads will be sent off course by phone companies like Google Translate.


1192
If you already have early birds then the large skulls can be used as guinea pigs for medical cloning in North America.


1192
If we simply ignore bad data simply the slow path can be explained with observations from signal function to signal strength for simple Poisson.


1192
While it currently holds strong positions already a good week has been covered in advance by pundits such as pundit Max Yasgur.


1193
Paul and Foggy, longtime lovers, who both were swept into the hospital by Merryn coming home from the hospital, then she met the morning after Pearl Harbor and she greeted the strangers with Good.


1193
Jon and Rob ( mental health ) what I was doing during a chat with Robin started just over a month, so we had a meeting with Robin Kligerman and we had a meeting with Zim.


1193
Waugh and Pierce ’ poor friendship : they both were discombobulated

In [31]:
df_pos_sentiment_tense = pd.DataFrame({'text': generated_sentences, 'pos': control_pos, 'sentiment': control_sentiment, 'tense': control_tense})
df_pos_sentiment_tense

,text,pos,sentiment,tense
0,It ’s useless.,PRON VERB ADJ PUNCT,negative,present
1,They had lost their entire expedition.,PRON AUX VERB PRON ADJ NOUN PUNCT,negative,past
2,It was painful.,PRON AUX ADJ PUNCT,negative,past
3,It is painted.,PRON AUX VERB PUNCT,negative,present
4,Based on this will hurt business lending insti...,VERB ADP PRON AUX VERB NOUN VERB NOUN NOUN PUN...,negative,future
...,...,...,...,...
995,"He ’s growing, out of growing to his max.",PRON AUX VERB PUNCT ADP ADP VERB ADP PRON NOUN...,positive,present
996,"And despite California ’s drought battling, mi...",CCONJ SCONJ PROPN NOUN NOUN VERB PUNCT NOUN AD...,positive,past
997,From the bench graphs above the reflected valu...,ADP DET NOUN VERB ADP DET VERB NOUN PUNCT ADJ ...,positive,present
998,Research on self - driving cars shows to the p...,NOUN ADP NOUN PUNCT VERB NOUN VERB ADP DET NOU...,positive,present


In [32]:
df_pos_sentiment_tense.to_csv('../../../Controllable-text-gen-with-DM/datasets/Generation_results/df_pos_sentiment_tense.csv', index=False)

## (rudimentary) Metrics

### len

In [23]:
# for 'len'
n_trials = 100
n_words = [5, 10, 15, 20, 25]
mse = []
for _ in range(n_trials):
    for nw in n_words:
        gen_str = generate(model, tokenizer, {'words': nw})
        nw_pred = find_attributes(gen_str)['words']
        mse.append(np.sqrt((nw - nw_pred)**2)/nw)
    
print(np.average(mse))

0.055766666666666666


epoch-checkpoint-6.0 : 0.488  | 0.04478666666666666 \
epoch-checkpoint-5.0 : 0.455 \
epoch-checkpoint-4.0 : 0.445  | 0.037046666666666665 \
epoch-checkpoint-3.0 : 0.575 \
epoch-checkpoint-2.0 : 0.9

### pos

In [122]:
def needleman_wunsch(seq1, seq2, match_score=1, mismatch_score=-1, gap_penalty=-1):
    """
    Very simple aligning algorithm.
    """

    m, n = len(seq1), len(seq2)
    score_matrix = np.zeros((m + 1, n + 1), dtype=int)

    for i in range(m + 1):
        score_matrix[i, 0] = i * gap_penalty
    for j in range(n + 1):
        score_matrix[0, j] = j * gap_penalty

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = score_matrix[i - 1, j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score)
            delete = score_matrix[i - 1, j] + gap_penalty
            insert = score_matrix[i, j - 1] + gap_penalty
            score_matrix[i, j] = max(match, delete, insert)

    aligned_seq1 = []
    aligned_seq2 = []
    i, j = m, n

    while i > 0 or j > 0:
        if i > 0 and j > 0 and score_matrix[i, j] == score_matrix[i - 1, j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score):
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and score_matrix[i, j] == score_matrix[i - 1, j] + gap_penalty:
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append('-')
            i -= 1
        else:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j - 1])
            j -= 1

    aligned_seq1 = list(reversed(aligned_seq1))
    aligned_seq2 = list(reversed(aligned_seq2))

    return aligned_seq1, aligned_seq2

Aligned Ground Truth: ['NN', 'VB', 'DT', 'NN']
Aligned Predicted: ['NN', 'VB', '-', 'NN']


In [139]:
np.random.seed(108)
pos_samples = df_val.sample(n=500)['pos'].values
pos_samples[:3]

array(['ADV PUNCT PRON AUX VERB ADP DET PROPN NOUN PUNCT',
       'CCONJ VERB DET NOUN VERB DET ADJ NOUN ADP DET PROPN PROPN SCONJ PRON VERB PRON PUNCT',
       'PRON VERB PART PART VERB ADP PRON PUNCT'], dtype=object)

In [150]:
matches = []
for pos_input in pos_samples:
    for nw in n_words:
        gen_str = generate(model, tokenizer, {'pos': pos_input})
        pos_pred = find_attributes(gen_str)['pos']
        aligned_input, aligned_pred = needleman_wunsch(pos_input.split(), pos_pred.split())
        matches += [g == p for g,p in zip(aligned_input, aligned_pred)]
print(np.average(matches))

0.9261392729134664


epoch-checkpoint-6.0 : 0.917 \
epoch-checkpoint-5.0 : 0.917 \
epoch-checkpoint-4.0 : 0.9261 